In [1]:
import pandas as pd
import numpy as np
import pdfplumber
import csv

In [2]:
def convert(pdf_path, csv_path):
    with pdfplumber.open(pdf_path) as pdf, open(csv_path, "w") as csv_file:
        writer = csv.writer(csv_file, lineterminator="\n")
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                if table:
                    for row in table:
                        writer.writerow(row)

In [3]:
convert("../statement_pdf.pdf", "../statement.csv")

In [6]:
df = pd.read_csv("../statement.csv", header=None)

In [7]:
df.head()

,0,1,2,3,4,5
0,DATE,DESCRIPTION,CHEQUE NO,DEBIT,CREDIT,BALANCE
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,NaN,30.00,NaN,"18,985.78"
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,NaN,17.00,NaN,"18,968.78"
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,NaN,32.00,NaN,"18,936.78"
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,NaN,47.50,NaN,"18,889.28"


In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.drop(0, axis=0, inplace=True)

In [10]:
df.head()

,0,1,2,3,4,5
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,NaN,30.00,NaN,"18,985.78"
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,NaN,17.00,NaN,"18,968.78"
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,NaN,32.00,NaN,"18,936.78"
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,NaN,47.50,NaN,"18,889.28"
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,NaN,NaN,1.00,"18,890.28"


In [11]:
def categorize(desc: str, debit):
    desc = str(desc).upper()
    if "UPI" in desc:
        return "Expenses:UPI"
    elif "INTEREST" in desc:
        return "Income:Interest"
    elif debit:
        return "Expenses:Misc"
    else:
        return "Income:Misc"

In [12]:
df.drop([2, 5], axis=1, inplace=True)

In [13]:
df.head()

,0,1,3,4
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,30.00,NaN
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,17.00,NaN
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,32.00,NaN
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,47.50,NaN
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,NaN,1.00


In [14]:
df.columns = ["Date", "Description", "Debit", "Credit"]

In [15]:
df.head()

,Date,Description,Debit,Credit
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,30.00,NaN
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,17.00,NaN
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,32.00,NaN
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,47.50,NaN
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,NaN,1.00


In [16]:
df['Account'] = df.apply(lambda x: categorize(x["Description"], x["Debit"]), axis=1)

In [17]:
df.head()

,Date,Description,Debit,Credit,Account
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,30.00,NaN,Expenses:UPI
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,17.00,NaN,Expenses:UPI
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,32.00,NaN,Expenses:UPI
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,47.50,NaN,Expenses:UPI
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,NaN,1.00,Expenses:UPI


In [18]:
df["Debit"] = df['Debit'].fillna(0)
df["Credit"] = df['Credit'].fillna(0)

In [19]:
def currency_to_float(value):
    if isinstance(value, str):
        value = value.replace('₹', '').replace('$', '')  # Remove currency symbols
        value = value.replace(',', '')                  # Remove commas
    return float(value)


In [20]:
df["Debit"] = df["Debit"].apply(currency_to_float)
df["Credit"] = df["Credit"].apply(currency_to_float)

In [21]:
df["Amount"] = df["Credit"] - df["Debit"]

In [22]:
df.head()

,Date,Description,Debit,Credit,Account,Amount
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,30.0,0.0,Expenses:UPI,-30.0
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,17.0,0.0,Expenses:UPI,-17.0
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,32.0,0.0,Expenses:UPI,-32.0
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,47.5,0.0,Expenses:UPI,-47.5
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,0.0,1.0,Expenses:UPI,1.0


In [23]:
df = df[["Date", "Description", "Account", "Amount"]]

In [24]:
df.head()

,Date,Description,Account,Amount
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,Expenses:UPI,-30.0
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,Expenses:UPI,-17.0
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,Expenses:UPI,-32.0
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,Expenses:UPI,-47.5
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,Expenses:UPI,1.0


In [25]:
df.to_csv("statement.csv", index=False)

In [26]:
df[df['Date'] != "TOTAL"]

,Date,Description,Account,Amount
1,03/04/2025,TO ONL UPI/DR/509335780211/THE ULTI/YESB/PAYTM...,Expenses:UPI,-30.0
2,04/04/2025,TO ONL UPI/DR/546087903003/INDIAN I/SBIN/38329...,Expenses:UPI,-17.0
3,04/04/2025,TO ONL UPI/DR/546078415439/INDIAN I/SBIN/38329...,Expenses:UPI,-32.0
4,06/04/2025,TO ONL UPI/DR/546320612982/SAPTARSI/SBIN/SAPTA...,Expenses:UPI,-47.5
5,07/04/2025,BY ONL UPI/CR/509734628153/KHURSHID/NSPB/AKURS...,Expenses:UPI,1.0
...,...,...,...,...
67,28/06/2025,TO ONL UPI/DR/517984203840/RS AGENC/YESB/PAYTM...,Expenses:UPI,-215.0
68,29/06/2025,TO ONL UPI/DR/518086972754/MAHESH S/YESB/PAYTM...,Expenses:UPI,-88.0
69,29/06/2025,TO ONL UPI/DR/518092952923/VIJAYA K/YESB/PAYTM...,Expenses:UPI,-10.0
70,30/06/2025,BY ONL UPI/CR/503670351815/GOOGLE P/UTIB/GOOGL...,Expenses:UPI,2.0


In [33]:
import re

def mask_desc(desc):
    return re.sub(r"\d", "x", str(desc))

In [34]:
df[df["Description"] == ""]

,Date,Description,Account,Amount


In [35]:
df['Desc_masked'] = df["Description"].apply(mask_desc)

In [36]:
df["Desc_masked"]

1     TO ONL UPI/DR/xxxxxxxxxxxx/THE ULTI/YESB/PAYTM...
2     TO ONL UPI/DR/xxxxxxxxxxxx/INDIAN I/SBIN/xxxxx...
3     TO ONL UPI/DR/xxxxxxxxxxxx/INDIAN I/SBIN/xxxxx...
4     TO ONL UPI/DR/xxxxxxxxxxxx/SAPTARSI/SBIN/SAPTA...
5     BY ONL UPI/CR/xxxxxxxxxxxx/KHURSHID/NSPB/AKURS...
                            ...                        
68    TO ONL UPI/DR/xxxxxxxxxxxx/MAHESH S/YESB/PAYTM...
69    TO ONL UPI/DR/xxxxxxxxxxxx/VIJAYA K/YESB/PAYTM...
70    BY ONL UPI/CR/xxxxxxxxxxxx/GOOGLE P/UTIB/GOOGL...
71                             BY CREDIT INTEREST:xxxxx
72                                                  nan
Name: Desc_masked, Length: 71, dtype: object

In [37]:
df.tail()

,Date,Description,Account,Amount,Desc_masked
68,29/06/2025,TO ONL UPI/DR/518086972754/MAHESH S/YESB/PAYTM...,Expenses:UPI,-88.00,TO ONL UPI/DR/xxxxxxxxxxxx/MAHESH S/YESB/PAYTM...
69,29/06/2025,TO ONL UPI/DR/518092952923/VIJAYA K/YESB/PAYTM...,Expenses:UPI,-10.00,TO ONL UPI/DR/xxxxxxxxxxxx/VIJAYA K/YESB/PAYTM...
70,30/06/2025,BY ONL UPI/CR/503670351815/GOOGLE P/UTIB/GOOGL...,Expenses:UPI,2.00,BY ONL UPI/CR/xxxxxxxxxxxx/GOOGLE P/UTIB/GOOGL...
71,30/06/2025,BY CREDIT INTEREST:99999,Income:Interest,138.00,BY CREDIT INTEREST:xxxxx
72,TOTAL,NaN,Expenses:Misc,370.74,nan


In [38]:
df["Desc_masked"].to_clipboard()